In [332]:
# 경쟁사 비교 에이전트

# 역할 : 경쟁 구도, 차별성 분석
# 내용 : 경쟁사 정보 검색 및 비교 분석

# 입력 : 기업명

In [333]:
# import

from langchain_teddynote.tools.tavily import TavilySearch

from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

from langchain_openai import ChatOpenAI
from langchain_teddynote.evaluator import GroundednessChecker
from langchain_teddynote.messages import messages_to_history

from langgraph.graph import END, StateGraph
from langgraph.checkpoint.memory import MemorySaver

from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import stream_graph, random_uuid
from langgraph.graph import StateGraph, START, END

from langchain.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.messages import BaseMessage
import os

In [334]:
#!pip install langchain-teddynote

In [336]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [337]:
# 그래프 상태
class GraphState(TypedDict):  
    question: Annotated[str, "Question"] # 질문
    context: Annotated[str, "Context"]    
    answer: Annotated[str, "Answer"]        # 답변  
    messages: Annotated[list[BaseMessage], "Messages"] # 메시지(누적되는 list)  
    relevance: Annotated[str, "Relevance"]  # 관련성  

In [339]:
def relevance_check(state: GraphState) -> GraphState:
    # LLM을 직접 사용하여 관련성 체크
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    
    # 관련성 체크 프롬프트
    prompt = f"""
    당신은 질문과 검색 결과의 관련성을 평가하는 평가자입니다.
    
    질문: {state["question"]}
    
    검색 결과: {state["context"]}
    
    위 질문과 검색 결과가 서로 관련이 있는지 판단해주세요. 
    관련성이 있다면 "yes"를, 없다면 "no"를 응답해주세요.
    오직 "yes" 또는 "no"로만 대답하세요.
    """
    
    # LLM 실행
    response = llm.invoke(prompt)
    result = response.content.strip().lower()
    
    # 결과 출력
    print("==== [RELEVANCE CHECK] ====")
    print(f"결과: {result}")
    
    # "yes"면 True, "no"면 False 반환
    is_relevant = result == "yes"
    return GraphState(relevance=is_relevant)

def is_relevant(state: GraphState) -> str:
    """조건부 라우팅을 위한 함수"""
    if state["relevance"]:
        return "yes"
    else:
        return "no"

In [340]:
def llm_answer(state: GraphState) -> GraphState:
    latest_question = state["question"]
    context = state["context"]  # 웹 검색 결과로 받은 경쟁사 분석 정보

    # 경쟁사 분석 결과를 요약하고 핵심 인사이트를 추출하는 프롬프트
    report_prompt = f"""
        다음은 {latest_question}에 대한 경쟁사 분석 정보입니다. 이 내용을 기반으로 요약 보고서를 작성해 주세요.

        형식:
        1. 경쟁사 리스트 정리
        2. 핵심 인사이트 정리
        3. 전략적 시사점
        4. 투자측면에서 이점

        경쟁사 분석 정보: {context}
        """

    response = llm.invoke(report_prompt)
    
    # 생성된 답변과 (유저의 질문, 답변) 메시지를 상태에 저장
    return GraphState(
        answer=response, 
        messages=[("user", latest_question), ("assistant", response)]
    )


In [342]:
def web_search(state: GraphState) -> GraphState:
    print(f"Received state: {state}")  # state 객체 확인
    
    tavily_tool = TavilySearch()
    company_name = state["question"]  # 'question' 키에서 기업명 추출
    search_query = f"{company_name} 경쟁사 비교 분석"
    print(f"Searching for: {search_query}")  # 검색 쿼리 확인
    
    # 최근 한 달의 결과 5개 가져오기
    search_result = tavily_tool.search(
        query=search_query,
        topic="general",    # 'general'로 변경 (유효한 옵션)
        days=30,            # 최근 한 달 데이터로 확장
        max_results=3,      # 결과 수 증가
        format_output=True, # 결과 포맷팅
    )

    if search_result:
        print(f"Search Results: {search_result}")  # 검색 결과 확인
        
        # 결과 헤더 추가하여 컨텍스트 명확히 하기
        formatted_result = f"## {company_name} 경쟁사 분석 정보\n\n" + "\n\n".join(search_result)
        return GraphState(context=formatted_result)
    else:
        print("No search results found.")
        return GraphState(context=f"{company_name}에 대한 경쟁사 비교 정보를 찾을 수 없습니다.")


In [343]:
# 워크 플로우
workflow = StateGraph(GraphState)

workflow.add_node("web_search", web_search)
workflow.add_node("relevance_check", relevance_check)
workflow.add_node("llm_answer", llm_answer)

workflow.add_edge(START, "web_search")
workflow.add_edge("web_search", "relevance_check")  
workflow.add_edge("llm_answer", END)  

workflow.add_conditional_edges(
    "relevance_check",  
    is_relevant,
    {
        "yes": "llm_answer", 
        "no": "web_search",  
    },
)

workflow.set_entry_point("web_search")

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [344]:
from langchain_teddynote.graphs import visualize_graph  

visualize_graph(app)  

[ERROR] Visualize Graph Error: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)


In [345]:
# 파라미터 입력 받기
config = RunnableConfig(recursion_limit=10, configurable={"thread_id": random_uuid()})

company = input("기업명을 입력하세요 : ")
# 수정된 GraphState 예시
inputs = GraphState(
    question=company,
    context="",  # context 비워두기
    answer="",   # 초기값 비워두기
    messages=[], # 빈 메시지 목록
    relevance="",  # 초기값 비워두기
    usage_metadata={}  # usage_metadata를 빈 딕셔너리로 추가
)

# stream_graph 호출
stream_graph(app, inputs, config, ["web_search", "relevance_check", "llm_answer"])

Received state: {'question': 'SK', 'context': '', 'answer': '', 'messages': [], 'relevance': ''}
Searching for: SK 경쟁사 비교 분석
Search Results: ['<document><title>SK텔레콤 분석리포트 2024 ver. ㅣ캐치 - CATCH</title><url>https://www.catch.co.kr/Comp/AnalysisCompView?ID=2251</url><content>sk텔레콤 분석리포트 2024 ver. 한눈에 보는 sk텔레콤 사업분석, 경쟁사비교, swot분석1984년 kmts라는 사명으로 설립된 sk텔레콤은 sk그룹의 계열사로, 세계 최초로 cdma(2g) 서비스를 시작한 ict 복합기업이다. 당사는 통신, 미디어, 보안, 커머스, 모빌리티 등 다양한 분야에서 사업을 영위하고 있</content></document>', '<document><title>SK - 경쟁사분석 - 기업모니터 - WiseReport</title><url>https://comp.wisereport.co.kr/samsung/company/c1060001.aspx?cmp_cd=034730</url><content>경쟁사분석: 업종분석: 지분현황 ... ※ 각 항목 입력 후 엔터키를 누르시면 분석결과를 확인하실수 있습니다. ※ 재무아이템을 클릭하시면 아래 기업의 최근 결산/분기 순액 데이터를 차트로 확인하실수 있습니다. 기업간비교자료(연간) 항목 sk a034730</content><raw>메인메뉴\n\n서브메뉴\n\n기업현황 | 기업개요 | 재무분석 | 투자지표 | 컨센서스 | 경쟁사분석 | 업종분석 | 지분현황 | 최근리포트 | 금감원공시\n기업모니터\n\n요약정보\n\n | SK034730홈페이지대표전화주소EPS-17,618BPS455,927PERN/A업종PERN/APBR0.28현금배당수익률5.50%전일종가127,300도움말KOSPI : 코스피 기타금융업|WI26